# Test

In [38]:
import random
import os
# notre package de fonctionnalités
from Package import Scripts_Analyse01 as pk
import pandas as pd
import numpy as np
import missingno
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats
from scipy.stats import pearsonr
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from yellowbrick.features import ParallelCoordinates
from plotly.graph_objects import Layout
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.features import PCA as PCA_yellow
from yellowbrick.style import set_palette
from sklearn import metrics
import gc
import time
from contextlib import contextmanager
import warnings
import numpy as np
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from dotenv import load_dotenv
from pprint import pprint
warnings.simplefilter(action='ignore', category=FutureWarning)
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [10]:
%flake8_off

In [24]:
# chargement des variables d'environnment
load_dotenv() 

Python-dotenv could not parse statement starting at line 1


True

In [40]:
api_key = os.getenv('API_KEY')
endpoint = "https://api.yelp.com/v3/graphql"

Réalisons une fonction pour réaliser des requêtes en GraphQL. On utilise cette solution afin de pouvoir collecter les informations uniquement des champs qui nous intéressent.

In [42]:
def request_client(api_key, endpoint, query):
    reqHeaders = {
        'Authorization': 'Bearer ' + api_key 
    }

    _transport = RequestsHTTPTransport(
        url=endpoint,
        headers = reqHeaders,
        use_json=True,
    )

    client = Client(
        transport = _transport,
        fetch_schema_from_transport=True,
    )

    result = client.execute(query)
    return result

In [178]:
# le max de review est de 3 par restaurant, on importe toutes les données
restaurants=[]
j = 0
for i in range(0,4):
    query = """
                {
                search(term:"restaurant",
                        location:"FR"
                        limit:50,
                        offset:"""+str(j)+"""
                        ) {
                    business {
                    id,
                    name,
                    photos,
                    reviews{
                        id,
                        text,
                        rating,
                        url
                    }
                    }
                }
                }
            """
    query = gql(query)
    temp = request_client(api_key, endpoint, query)["search"]["business"]
    restaurants.extend(temp)
    print("nb restaurants ", len(restaurants))
    j=j+51
    

nb restaurants  50
nb restaurants  100
nb restaurants  150
nb restaurants  200


In [183]:
pprint(pd.DataFrame(restaurants))

                         id                                 name  \
0    -0iLH7iQNYtoURciDpJf6w        Le Comptoir de la Gastronomie   
1    IU9_wVOGBKjfqTTpAXpKcQ                 Bistro des Augustins   
2    KggnM_Z4wOa_JExunaaWHg                 Le Temps des Cerises   
3    cEjF41ZQB8-SST8cd3EsEw                     L'Avant Comptoir   
4    ctP4c3mwVO5oOzLI48LtuQ                      Les Antiquaires   
..                      ...                                  ...   
195  d_tt3AgHNkphyH-fuI0e_g                         Le Rigmarole   
196  OrggqA6CfhQmO3UrQr1K0w                            Lapérouse   
197  B1LO7k6XfAGXLQZ0wCk_3w                     Le Vin en Bouche   
198  -YKc0e0z_47s8JeXA2feOg  La Tour de Montlhéry ou chez Denise   
199  JAvCagWNHVKEi-MbpvEcHw                 Le Potager du Marais   

                                                photos  \
0    [https://s3-media2.fl.yelpcdn.com/bphoto/TtMQl...   
1    [https://s3-media4.fl.yelpcdn.com/bphoto/J9RBy...   
2    [htt

On scinde notre fichier en deux afin de traiter le texte et les photos séparément.

Analysons le texte en premier lieu

In [9]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

text = "La seconde partie du cours de traitement de texte traite de la transformation des données textuelles"
token = nltk.word_tokenize(text)
trigrams = ngrams(token,3)
print(len(list(trigrams)))

14


In [ ]:
import requests
# this is the url for Alice in Wonderland
result = requests.get('http://www.gutenberg.org/files/11/11-0.txt')
print(result.text)